Models

In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [7]:
# Linear model
scaled_data = pd.read_csv('info_data.csv')
originaldata = pd.read_csv('../Data/Venta_Consumidor_Producto_UPC_PrevVTA_2018.csv')

dataset = originaldata.drop(columns=['ANO_MES_FACTURA', 'CANAL_VENTA_DESC', 'TEMPORADA_COMERCIAL_DESC', 'MATERIAL_ID', 'ESFUERZO_VENTA_DESC', 'NUMERO_DEUDOR', 'NUMERO_DEUDOR_PAIS_DESC', 'GRUPO_ARTICULO', 'CONCEPTO', 'LINEA',  'PAIS_CONTACTO', 'PAIS_CONTACTO_DESC'])
cleanedataset = dataset[(dataset.CANAL_VENTA_ID == 20) & (dataset.FACTURA_CLASE_DOCUMENTO_ID == 'ZTON')]
cleanedataset = cleanedataset[(cleanedataset.IMP_VENTA_NETO_EUR.isnull() == False)]
cleanedataset.fillna("NV", inplace = True)